In [63]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import re
import selenium
import pandas as pd
import html5lib
import math


In [194]:
"""
start_time = time.time()
url = 'https://www.ratemyprofessors.com/search/teachers?query=*&sid=1222'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)


driver.find_element_by_class_name('Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 eaZArN').click()

#hej=driver.find_element_by_class_name('SearchResultsPage__AddPromptWrapper-sc-1srop1v-2 gUWHGC').click()
#driver.find_element_by_class_name('CardName__StyledCardName-sc-1gyrgim-0 cJdVEK')
driver.find_element(By.linkText("Show More")).click()




#soup = BeautifulSoup(driver.page_source, 'lxml')
#company=soup.find('td',{'id':'domain_registrant_name'}).text.strip()
#print(company)
"""

'\nstart_time = time.time()\nurl = \'https://www.ratemyprofessors.com/search/teachers?query=*&sid=1222\'\ndriver = webdriver.Chrome(ChromeDriverManager().install())\ndriver.get(url)\n\n\ndriver.find_element_by_class_name(\'Buttons__Button-sc-19xdot-1 PaginationButton__StyledPaginationButton-txi1dr-1 eaZArN\').click()\n\n#hej=driver.find_element_by_class_name(\'SearchResultsPage__AddPromptWrapper-sc-1srop1v-2 gUWHGC\').click()\n#driver.find_element_by_class_name(\'CardName__StyledCardName-sc-1gyrgim-0 cJdVEK\')\ndriver.find_element(By.linkText("Show More")).click()\n\n\n\n\n#soup = BeautifulSoup(driver.page_source, \'lxml\')\n#company=soup.find(\'td\',{\'id\':\'domain_registrant_name\'}).text.strip()\n#print(company)\n'

In [195]:
"""
school_id=1222
number_of_professors=464
number_of_pages=math.ceil(number_of_professors/20)
school_url="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)



#VIGTIG_URL="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=1222"
#20 PROFESSORER PR SIDE
#464 professors at Yale University

response = requests.get(school_url)
list_of_dicts=response.json() #keys are: ['professors', 'searchResultsTotal', 'remaining', 'type']
list_of_professor_dicts=list_of_dicts['professors'] 
number_of_professors=list_of_dicts['searchResultsTotal']

real_df=pd.concat([pd.DataFrame(list_of_dicts['professors'] [i], index=[i]) for i in range(20)],
          ignore_index=True)

"""

'\nschool_id=1222\nnumber_of_professors=464\nnumber_of_pages=math.ceil(number_of_professors/20)\nschool_url="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)\n\n\n\n#VIGTIG_URL="https://www.ratemyprofessors.com/filter/professor/?&page=1&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid=1222"\n#20 PROFESSORER PR SIDE\n#464 professors at Yale University\n\nresponse = requests.get(school_url)\nlist_of_dicts=response.json() #keys are: [\'professors\', \'searchResultsTotal\', \'remaining\', \'type\']\nlist_of_professor_dicts=list_of_dicts[\'professors\'] \nnumber_of_professors=list_of_dicts[\'searchResultsTotal\']\n\nreal_df=pd.concat([pd.DataFrame(list_of_dicts[\'professors\'] [i], index=[i]) for i in range(20)],\n          ignore_index=True)\n\n'

The keys for each element in list_of_professor_dicts are:

'tDept' = Department (eg. science, spanish, law ect)

'tSid' = The school id (eg. 1222 for all Yale professors)

'institution_name' = Name of university

'tFname' = First name

'tMiddlename' = Middle name

'tLname' = Last name

'tid' = Teacher id

'tNumRatings' = Number of ratings

'rating_class' = How is their score (eg. good, average, poor, zero)

'contentType' = Have only seen this to return 'TEACHER'

'categoryType'= Have only seen this to return 'PROFESSOR'

'overall_rating' = Average rating (float)

In [182]:
def get_dataframe_of_school(school_id, number_of_professors):
    """This function returns a dataframe with al teachers for a given school as well as a bunch of attributes"""
    number_of_pages=math.ceil(number_of_professors/20)
    list_of_dfs=[]
    for j in range(1,number_of_pages+1):
        url=f"https://www.ratemyprofessors.com/filter/professor/?&page={j}&filter=teacherlastname_sort_s+asc&query=*%3A*&queryoption=TEACHER&queryBy=schoolId&sid="+str(school_id)
        response = requests.get(url)
        list_of_dicts=response.json() #keys are: ['professors', 'searchResultsTotal', 'remaining', 'type']
        list_of_professor_dicts=list_of_dicts['professors'] 
        list_of_dfs.append(pd.concat([pd.DataFrame(list_of_dicts['professors'] [i], index=[i]) for i in range(len(list_of_professor_dicts))],
                  ignore_index=True))
    return pd.concat([list_of_dfs[i] for i in range(len(list_of_dfs))], axis= 0).reset_index(drop=True)

        
hej=get_dataframe_of_school(1222, 464)

In [190]:
def add_comments_to_df(df):
    list_of_teacher_ids=list(df.tid)
    list_of_comments_for_each_teacher=[]
    for i in range(len(list_of_teacher_ids)):
        print(i)
        my_url = f'https://www.ratemyprofessors.com/ShowRatings.jsp?tid={list_of_teacher_ids[i]}'
        my_response = requests.get(my_url)
        html = my_response.text
        soup = BeautifulSoup(html,'lxml')
        my_text = soup.findAll('div', {'class':'Comments__StyledComments-dzzyvm-0 gRjWel'})#list of comments unparsed
        finished_comments=[i.text for i in my_text]
        list_of_comments_for_each_teacher.append(finished_comments)
    df['Comments']=list_of_comments_for_each_teacher
        
    return df

ok=add_comments_to_df(hej)
ok

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
